本章我们要讨论一种
简单的非线性模型，用来解决回归与分类问题，称为决策树（decision tree）。首先，我们将用决策
树做一个广告屏蔽器，可以将网页中的广告内容屏蔽掉。之后，我们介绍集成学习（lensemble
learning）方法，通过将一系列学习方法集成使用，以取得更好的训练效果

数据集：http://archive.ics.uci.edu/ml/datasets/Internet+Advertisements

不过类型的比例并不协调，459幅广告图片，2820幅正常内容。决
策树学习算法可以从比例并不协调的数据集中生成一个不平衡的决策树（biased tree）。在决定是否
值得通过过抽样（over-sampling）和欠抽样（under-sampling）的方法平衡训练集之前，我们将用
不相关的数据集对模型进行评估。本例的解释变量就是图片的尺寸，网址链接里的单词，以及图片标
签周围的单词。响应变量就是图片的类型。解释变量已经被转换成特征向量了。前三个特征值表示宽
度，高度，图像纵横比（aspect ratio）。剩下的特征是文本变量的二元频率值。下面，我们用网格
搜索来确定决策树模型最大最优评价效果（F1 score）的超参数，然后把决策树用在测试集进行效果
评估

In [5]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

import zipfile
z = zipfile.ZipFile('./some datasets/ad-dataset.zip')
df = pd.read_csv(z.open(z.namelist()[0]),header=None,low_memory=False)
explanatory_variable_columns = set(df.columns.values)
response_variable_column = df[len(df.columns.values)-1]
explanatory_variable_columns.remove(len(df.columns.values)-1)

y = [1 if e=='ad.' else 0 for e in response_variable_column]
X = df.loc[:,list(explanatory_variable_columns )]

X.replace(to_replace=' *\?',value=-1,regex=True,inplace=True)
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [9]:
pipeline = Pipeline([
    ('clf',DecisionTreeClassifier(criterion='entropy'))
])

In [10]:
parameters = {
    'clf_max_depth':(150,155,160),
    'clf_min_samples_split':(1,2,3),
    'clf_min_samples_leaf':(1,2,3)
}

In [11]:
grid_search = GridSearchCV(pipeline,parameters,n_jobs=-1,verbose=1,scoring='f1')
grid_search.fit(X_train,y_train)
print('Best score: %0.3f' % grid_search.best_score_)
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t%s: %r' % (param_name,best_parameters[param_name]))
predictions = grid_search.predict(X_test)
print(classification_report(y_test,predictions))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\Kismet\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...da3\\lib\\site-packages\\ipykernel\\__main__.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\Kismet\Anaconda3\lib\runpy.

# 决策树集成

集成学习方法将一堆模型组合起来使用，比单个模型可以获取更好的效果。随机森林（random
forest）是一种随机选取训练集解释变量的子集进行训练，获得一系列决策树的集合的方法。随机森
林通常用其决策树集合里每个决策树的预测结果的均值或众数作为最终预测值。scikit-learn里的随机
森林使用均值作为预测值。随机森林相比单一决策树，不太会受到拟合过度的影响，因为随机森林的
每个决策树都看不到训练集的全貌，只是训练一部分解释变量数据，不会记忆训练集的全部噪声。
下面我们用随机森林升级我们的广告屏蔽程序。把前面用的DecisionTreeClassifier替换
成RandomForestClassifier就可以了。和前面一样，我们仍然用网格搜索来探索最优超参数

In [12]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

import zipfile
z = zipfile.ZipFile('./some datasets/ad-dataset.zip')
df = pd.read_csv(z.open(z.namelist()[0]),header=None,low_memory=False)
explanatory_variable_columns = set(df.columns.values)
response_variable_column = df[len(df.columns.values)-1]
explanatory_variable_columns.remove(len(df.columns.values)-1)

y = [1 if e=='ad.' else 0 for e in response_variable_column]
X = df.loc[:,list(explanatory_variable_columns )]
X.replace(to_replace=' *\?',value=-1,regex=True,inplace=True)
X_train,X_test,y_train,y_test = train_test_split(X,y)

pipeline = Pipeline([
('clf', RandomForestClassifier(criterion='entropy'))
])
parameters = {
'clf__n_estimators': (5, 10, 20, 50),
'clf__max_depth': (50, 150, 250),
'clf__min_samples_split': (1, 2, 3),
'clf__min_samples_leaf': (1, 2, 3)
}

grid_search = GridSearchCV(pipeline,parameters,n_jobs=-1,verbose=1,scoring='f1')
grid_search.fit(X_train,y_train)
print('Best score: %0.3f' % grid_search.best_score_)
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t%s: %r' % (param_name,best_parameters[param_name]))
predictions = grid_search.predict(X_test)
print(classification_report(y_test,predictions))

Fitting 3 folds for each of 108 candidates, totalling 324 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\Kismet\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...da3\\lib\\site-packages\\ipykernel\\__main__.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\Kismet\Anaconda3\lib\runpy.